# Single Agent Demo (Python)

This notebook demonstrates how to interact with a **single agent** using the Python Agent Framework backend API.

## What You'll Learn
- How to get available agents
- How to send messages to a single agent
- How to handle responses
- Different response formats (user-friendly vs detailed)

## Prerequisites
Make sure the backend API is running on `http://localhost:8000`

```bash
cd Backend/python
python -m uvicorn main:app --reload
```

## Setup - Install Required Packages

In [1]:
# Install required packages
%pip install requests python-dotenv -q

import requests
import json
from datetime import datetime
from typing import Dict, Any, Optional, List
import time

print("? Packages installed and modules imported successfully!")

Note: you may need to restart the kernel to use updated packages.
? Packages installed and modules imported successfully!



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Configure API Connection

In [2]:
# API Configuration
API_BASE_URL = "http://localhost:8000"
TIMEOUT = 120  # 2 minutes

# Create session for connection pooling
session = requests.Session()

print(f"? Connected to API: {API_BASE_URL}")

? Connected to API: http://localhost:8000


## Test API Connection

Let's first verify that the backend API is running and accessible.

In [3]:
# Quick test to verify API is accessible
try:
    response = session.get(f"{API_BASE_URL}/health", timeout=5)
    print(f"? API Status: {response.status_code}")
    
    if response.status_code == 200:
        print("? Backend API is running and accessible!")
        health_data = response.json()
        print(f"   Status: {health_data.get('status')}")
    else:
        print(f"??  API returned status: {response.status_code}")
        print("Make sure the backend is running with: python -m uvicorn main:app --reload")
except requests.exceptions.RequestException as ex:
    print(f"? Cannot connect to API: {ex}")
    print("Please start the backend API first:")
    print("   cd Backend/python")
    print("   python -m uvicorn main:app --reload")

? API Status: 200
? Backend API is running and accessible!
   Status: healthy


## Step 1: Get Available Agents

In [4]:
# Get all available agents
response = session.get(f"{API_BASE_URL}/agents")
agents_data = response.json()

print("?? Available Agents:")
print("="*50)

for agent in agents_data["agents"]:
    name = agent["name"]
    agent_type = agent["type"]
    provider = agent.get("provider", "N/A")
    
    print(f"\n?? Agent: {name}")
    print(f"   Type: {agent_type}")
    print(f"   Provider: {provider}")
    
    if "capabilities" in agent and agent["capabilities"]:
        capabilities = ", ".join(agent["capabilities"])
        print(f"   Capabilities: {capabilities}")

print(f"\n? Total available agents: {agents_data['total']}")

?? Available Agents:

?? Agent: azure_openai_agent
   Type: azure_openai_agent
   Provider: azure_openai
   Capabilities: General assistance

?? Agent: ms_foundry_people_agent
   Type: ms_people_agent
   Provider: azure_foundry
   Capabilities: General assistance

?? Agent: bedrock_agent
   Type: bedrock
   Provider: aws_bedrock
   Capabilities: AWS Bedrock powered responses, Amazon foundation models, General question answering, Advanced AI capabilities

?? Agent: openai_agent
   Type: openai_agent
   Provider: azure_openai
   Capabilities: General assistance

? Total available agents: 4


## Step 2: Send a Simple Message to a Single Agent (User-Friendly Format)

In [5]:
# Prepare chat request for a single agent
chat_request = {
    "message": "What is artificial intelligence and how is it used in healthcare?",
    "agents": ["azure_openai_agent"],  # Single agent (align with .NET demo)
    "session_id": None,  # New session
    "format": "user_friendly"  # Default format
}

print("?? Sending message to single agent...")
print(f"Message: {chat_request['message']}")
print(f"Agent: {chat_request['agents'][0]}\n")

# Send the request
response = session.post(
    f"{API_BASE_URL}/chat",
    json=chat_request,
    timeout=TIMEOUT
)
response.raise_for_status()
result = response.json()

# Display response
print("\n?? Response:")
print("="*50)
print(f"Agent: {result['agent']}")
print(f"Session ID: {result['session_id']}")
print(f"\nContent:\n{result['content']}")

# Display metadata
if "metadata" in result and result["metadata"]:
    metadata = result["metadata"]
    print(f"\n?? Metadata:")
    print(f"   Format: {metadata.get('response_type', 'N/A')}")
    print(f"   Agent Count: {metadata.get('agent_count', 'N/A')}")
    print(f"   Total Turns: {metadata.get('total_turns', 'N/A')}")

?? Sending message to single agent...
Message: What is artificial intelligence and how is it used in healthcare?
Agent: azure_openai_agent


?? Response:
Agent: azure_openai_agent
Session ID: b0c55ebd-0b86-4e3d-bf8b-0f8ab22183d4

Content:
**Artificial intelligence (AI)** is a branch of computer science focused on creating systems that can perform tasks that normally require human intelligence. These tasks include learning, reasoning, problem-solving, perception, understanding language, and decision-making. AI technologies often rely on machine learning, where systems improve their performance by analyzing data and recognizing patterns.

**Uses of AI in Healthcare**

AI is transforming healthcare in several important ways:

1. **Diagnostics:**  
   AI systems can analyze medical images (like X-rays, MRIs, and CT scans) to detect diseases such as cancer, pneumonia, or fractures with high accuracy. For example, AI algorithms can help radiologists identify abnormalities faster and more rel

## Step 3: Send a Message with Detailed Format

The detailed format provides more information including individual agent responses, turn information, and metadata.

In [6]:
# Request with detailed format (single agent)
detailed_request = {
    "message": "Explain machine learning in simple terms.",
    "agents": ["azure_openai_agent"],  # single agent name
    "format": "detailed"
}

print("?? Sending message with DETAILED format...")
print(f"Message: {detailed_request['message']}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=detailed_request,
    timeout=TIMEOUT
)
response.raise_for_status()
detailed_result = response.json()

# Display detailed response (single-agent structure)
print("\n?? Detailed Response:")
print("="*50)
print(f"Agent: {detailed_result.get('agent', 'unknown')}")
print(f"Session ID: {detailed_result.get('session_id', 'N/A')}")
print(f"Timestamp: {detailed_result.get('timestamp', 'N/A')}")
print(f"Processing Time: {detailed_result.get('processing_time_ms', 0)} ms")

# Content
print(f"\n?? Content:\n{detailed_result.get('content', '')}")

# Usage
usage = detailed_result.get("usage") or {}
if usage:
    print("\n?? Token Usage:")
    print(f"   Prompt Tokens: {usage.get('prompt_tokens', 0)}")
    print(f"   Completion Tokens: {usage.get('completion_tokens', 0)}")
    print(f"   Total Tokens: {usage.get('total_tokens', 0)}")

# Metadata
metadata = detailed_result.get("metadata") or {}
if metadata:
    print("\n?? Metadata:")
    print(f"   Response Type: {metadata.get('response_type', 'single_agent')}")
    print(f"   Agent Count: {metadata.get('agent_count', 1)}")

print(f"\n?? Format: {detailed_result.get('format', 'user_friendly')}")

?? Sending message with DETAILED format...
Message: Explain machine learning in simple terms.


?? Detailed Response:
Agent: azure_openai_agent
Session ID: f0c149dc-c2e6-4188-84b0-8de90642b738
Timestamp: 2025-12-10T18:29:27.684463
Processing Time: 2630 ms

?? Content:
Machine learning is a way for computers to learn from experience, similar to how humans do. Instead of being given exact instructions for every task, a computer is given lots of examples (data) and uses them to find patterns or make decisions.

**For example:**  
Imagine you want a computer to tell the difference between pictures of cats and dogs. Instead of programming all the features of cats and dogs, you show the computer many pictures labeled "cat" or "dog." The computer looks for patterns in these pictures and learns to guess which animal is in a new, unlabeled image.

**In short:**  
Machine learning is teaching computers to make predictions or decisions by learning from data, rather than following only fixed rules

## Step 4: Auto-select Agent (No Agent Specified)

If you don't specify an agent, the system will automatically select one for you.

In [7]:
# Request without specifying an agent
auto_request = {
    "message": "Tell me a fun fact about space.",
    # No agents specified - will be auto-selected
    "format": "user_friendly"
}

print("?? Sending message with AUTO-SELECTED agent...")
print(f"Message: {auto_request['message']}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=auto_request,
    timeout=TIMEOUT
)
response.raise_for_status()
auto_result = response.json()

print("\n?? Response:")
print("="*50)
print(f"Auto-selected Agent: {auto_result['agent']}")
print(f"\nContent:\n{auto_result['content']}")

?? Sending message with AUTO-SELECTED agent...
Message: Tell me a fun fact about space.


?? Response:
Auto-selected Agent: azure_openai_agent

Content:
Here’s a fun fact: If you could fit Saturn into a large enough pool of water, it would float! Saturn is the least dense planet in our solar system—its average density is lower than that of water.


## Step 5: Using Long-Running Memory (Optional)

Enable memory to have the agent remember your name and persona across messages.

In [8]:
# Enable memory to personalize responses
print("?? Testing Long-Running Memory")
print("="*50 + "\n")

# First message - introduce yourself
memory_msg1 = {
    "message": "Hi! My name is Alex and I'm a software developer.",
    "agents": ["azure_openai_agent"],  # updated agent name
    "enable_memory": True,  # Enable memory!
    "format": "user_friendly"
}

print(f"[Turn 1] ?? {memory_msg1['message']}")
print(f"         Memory: ON ??\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=memory_msg1,
    timeout=TIMEOUT
)
mem_result1 = response.json()
mem_session_id = mem_result1.get("session_id")

print(f"?? {mem_result1.get('content', mem_result1)}\n")
print(f"? Session ID: {mem_session_id}")
print(f"? Memory extracted: Name='Alex', Persona='software developer'\n")

time.sleep(1)

# Follow-up - agent should remember
memory_msg2 = {
    "message": "What's my name and what do I do?",
    "session_id": mem_session_id,
    "agents": ["azure_openai_agent"],  # updated agent name
    "enable_memory": True,
    "format": "user_friendly"
}

print(f"[Turn 2] ?? {memory_msg2['message']}")
print(f"         Memory: ON ??\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=memory_msg2,
    timeout=TIMEOUT
)
mem_result2 = response.json()

print(f"?? {mem_result2.get('content', mem_result2)}\n")
print(f"? The agent remembered your information!")
print(f"\n?? Learn more in the 04-LongRunningMemory-Demo.ipynb notebook")

?? Testing Long-Running Memory

[Turn 1] ?? Hi! My name is Alex and I'm a software developer.
         Memory: ON ??

?? Hi Alex! It’s great to meet you. As a software developer, if you have any questions about programming, tools, frameworks, troubleshooting, or anything else, feel free to ask. What are you working on these days?

? Session ID: 9b80dbe4-175c-4177-9e38-0d5f76e080cd
? Memory extracted: Name='Alex', Persona='software developer'

[Turn 2] ?? What's my name and what do I do?
         Memory: ON ??

?? Your name is Alex, and you’re a software developer. If you’d like to share more about your work or interests, I’m here to help with anything you need!

? The agent remembered your information!

?? Learn more in the 04-LongRunningMemory-Demo.ipynb notebook


## Step 6: Working with Different Agent Types

Try sending messages to different types of agents to see how they respond differently.

In [9]:
# Try preferred agents (aligned with .NET demo) but only if available
agents_response = session.get(f"{API_BASE_URL}/agents", timeout=TIMEOUT)
agents_response.raise_for_status()
agents_payload = agents_response.json()
available_agents = {agent.get("name") for agent in agents_payload.get("agents", []) if agent.get("name")}

preferred_order = ["azure_openai_agent", "ms_foundry_people_agent", "bedrock_agent", "openai_agent"]
agent_types = [a for a in preferred_order if a in available_agents]

if not agent_types:
    agent_types = list(available_agents)

if not agent_types:
    raise ValueError("No agents returned from the API")

print(f"? Agents to test: {', '.join(agent_types)}")

for agent_type in agent_types:
    try:
        print(f"\n?? Testing agent: {agent_type}")
        print("="*50)
        
        test_request = {
            "message": "Hello! What can you help me with?",
            "agents": [agent_type],
            "format": "user_friendly"
        }
        
        response = session.post(
            f"{API_BASE_URL}/chat",
            json=test_request,
            timeout=TIMEOUT
        )
        response.raise_for_status()
        
        test_result = response.json()
        content = test_result.get("content", "")
        truncated_content = content[:200] + "..." if len(content) > 200 else content
        
        print(f"? Agent responded: {truncated_content or '(no content returned)'}")
    except requests.exceptions.RequestException as ex:
        print(f"??  Agent not available or error occurred: {ex}")
    except Exception as ex:
        print(f"? Error testing {agent_type}: {ex}")

? Agents to test: azure_openai_agent, ms_foundry_people_agent, bedrock_agent, openai_agent

?? Testing agent: azure_openai_agent
? Agent responded: Hello! I can help you with a wide range of tasks, including:

- **Answering questions** on various topics like science, history, technology, and more.
- **Providing explanations** for concepts or proc...

?? Testing agent: ms_foundry_people_agent
? Agent responded: I can assist you with a wide range of tasks, including:

- Answering questions or providing information on various topics
- Summarizing or analyzing documents you provide
- Helping with writing, editi...

?? Testing agent: bedrock_agent
? Agent responded: Hello! I can assist you with a variety of HR-related topics. Here's what I can help with:

1. **Company Policies:**
   - Overview of specific policies
   - Guidance on understanding policies

2. **Ben...

?? Testing agent: openai_agent
? Agent responded: Hello! I can help you with a wide range of technical and software developme

## Summary

In this notebook, you learned:
- ? How to retrieve available agents from the API
- ? How to send messages to a single agent
- ? The difference between user-friendly and detailed response formats
- ? How to enable long-running memory with `enable_memory` flag
- ? How to use auto-agent selection
- ? How to work with different agent types

## Key Features

### Response Formats
- **user_friendly**: Clean, synthesized response (recommended for most cases)
- **detailed**: Full conversation history with metadata (useful for debugging)

### Memory Feature (NEW)
- Enable with `enable_memory: True` in request
- Agent extracts and remembers your name and persona
- Personalizes future responses based on remembered information
- See **04-LongRunningMemory-Demo.ipynb** for complete examples

## Next Steps
- Try the **02-MultipleAgents-Demo** notebook to see how agents collaborate
- Explore the **03-ContentSafety-Demo** for content moderation features
- Check out the **04-LongRunningMemory-Demo** for session management and memory features